# Therapeutic Inertia Nano Sim Test

This notebook is designed to be a nanosim testing different therapeutic inertia parameters in order to see what might be the most effective for use in the CVD sim. 

Models: 
1. Using uncorrelated therapeutic inertia, get some people on meds (one level) + find "ideal" med value from NHANES. Assume everyone in nanosim should be on meds 
2. Implement mixed propensity option 
3. Add in multiple treatment options, create the ladder 
4. Add in SBP levels + effect of treatment

In [1]:
import pandas as pd, numpy as np, os
import scipy.stats, random, matplotlib.pyplot as plt 

## Data Loading / Creation

Model 1: 
Ideal medicated level from NHANES and therapeutic inertia value 

Model 2: 
Mixed propensity value 

Model 3: 
Treatment options

Model 4:
SBP exposure and medication effects

In [91]:
inertia = 0.25 ## Means 25% of people DO GET treated on any time step
medication_rate = 0.597492 # Approx 60% of people who should be on meds are on meds 
sim_prop_rate = 0.5 #Means 20% of the propensity is due to the sim and 80% is random 
churn_rate = 0.05 #Adding a "churn" which is meant to represent people dying + new people needing SBP meds (aging in basically) 

## The Nanosim

Model 1: Sim ID, Time Step, Medication (Yes/No)

In [89]:
def run_nanosim(pop, n_steps):
    final_data = pd.DataFrame()
    final_data['time_step'] = range(n_steps)
    final_data['percent_meds'] = range(n_steps)
    
    sims = pd.DataFrame()
    sims['ID'] = range(pop)
    sims['meds'] = 0 
    sims['sim_propensity'] = np.random.normal(0, sim_prop_rate, pop) 
    for i in range(n_steps):
        sims['time_step'] = i
        sims['random_propensity'] = np.random.normal(0, (1-sim_prop_rate), pop) 
        sims['total_propensity'] = scipy.stats.norm.cdf(sims['sim_propensity'] + sims['random_propensity'])

        #Reset some people to no meds due to "churn"
        chosen_idx = np.random.choice(pop, int(churn_rate*pop), replace = False)
        sims.iloc[chosen_idx, sims.columns.get_loc('meds')] = 0
        
        #Put some people on medication 
        sims['meds'] = np.where(sims['meds'] == 1, 1, np.where(sims['total_propensity'] < inertia, 1, 0))
        
        #Get percent medicated for each time step 
        final_data.iloc[i,1] = len(sims.loc[sims['meds'] == 1])/len(sims)
        
    return final_data

In [92]:
test = run_nanosim(1_000, 30)
test

,time_step,percent_meds
0,0,0.163
1,1,0.256
2,2,0.333
3,3,0.375
4,4,0.413
5,5,0.449
6,6,0.461
7,7,0.478
8,8,0.491
9,9,0.508
